In [15]:
import bs4 as bs
import urllib.request
import pandas as pd
import numpy as np
import re
import progressbar
from itertools import chain
from multiprocessing import Pool
import functools
import csv
import requests
from joblib import Parallel, delayed

In [16]:
def remove_duplicates(l):
    return list(set(l))

In [17]:
def flatten(list_):
    if any(isinstance(i, list) for i in list_):
        list_ = chain.from_iterable(list_)
    return remove_duplicates(list_)

In [18]:
def get_links(htmls, pattern):
    if isinstance(htmls, str):
        htmls = [htmls]
    container = []
    for i in htmls:
        source = urllib.request.urlopen(i)
        soup = bs.BeautifulSoup(source, 'lxml', parse_only=bs.SoupStrainer('a'))

        for link in soup.find_all('a', href=True):
            l = link['href']
            if re.match(pattern, l):
                container.append("http://www.dfshop.com"+l)
    container = flatten(container)
    return container

In [19]:
def get_category(html, pattern):
    pool = Pool(4)
    iter_cat = functools.partial(get_links, pattern=pattern)

    parents            = get_links(html, pattern=pattern)
    children           = pool.map(iter_cat, parents)
    #grandchildren      = flatten(pool.map(iter_cat, children))
    #ggchildren         = flatten(pool.map(iter_cat, grandchildren))

    return children

In [20]:
def get_article(html, pattern):
    pool = Pool(6)
    iter_art = functools.partial(get_links, pattern=pattern)

    arts               = flatten(pool.map(iter_art, ggchildren))
    return arts

In [31]:
source = urllib.request.urlopen("http://www.dfshop.com/TeamShop/")
soup = bs.BeautifulSoup(source, 'lxml', parse_only=bs.SoupStrainer('td'))

g = [i[0]+1 for i in enumerate(soup) if re.match(".+Sabag.+", str(i[1]))]
link = [i[1] for i in enumerate(soup) if i[0] == g[0]]
html = [i['href'] for i in link[0].find_all("a")]

In [32]:
s = requests.Session()
r = s.get(html[0])
soup = bs.BeautifulSoup(r.text, "lxml")
html = ["http://www.dfshop.com"+i['href'] for i in soup.find_all("a")]
html

['http://www.dfshop.com/dfshop/wsTeamShop.wsc/pTsHome.p?session=dTIWcvCibibjcbcdIdluizjmlcajaFsq']

In [ ]:
items = {}
total = 999999
for i in range(0, total):
    lenght = len(str(i))
    leading = '000000'[0:-lenght]
    request = html[0]+"&f=art&art=AA"+leading+str(i)
    try:
        r = s.get(request)
        #r = s.get(html[0]+"&f=art&art=AA"+"111004")
        soup = bs.BeautifulSoup(r.text, "lxml", parse_only=bs.SoupStrainer('div'))
        article = [i.text for i in soup.select("#artDetailArnumCell")]
        if len(article[0]) < 3:
            continue
        text = [i.text for i in soup.select("#artDetailDescCell")]
        price = [i.text for i in soup.select("#artDetailPriceCell")]
        price = re.findall("\d+\.\d+SFr", price[0])[0][0:-3]
        items[article[0]] = [text[0], price]
    except:
        continue

In [43]:
items

{'3111001': ['Badewanne NORM CLASSIC140 x 70 x 44 cm, StahlSchallisolierung',
  '609.00']}